In [1]:
from proteus.iproteus import * 
from proteus import default_n, default_s, default_so
Profiling.logLevel=5
Profiling.verbose=False

In [2]:
import variable_density2D.guermond_example_variable_density_so as so
display_src(so)

NameError: name 'physics' is not defined

In [ ]:
from proteus import Comm
import os
from importlib import import_module

pList=[]
nList=[]
so.sList=[]
for (p,n) in so.pnList:
    so.sList.append(default_s)
    pList.append(import_module('variable_density.'+p))
    nList.append(import_module('variable_density.'+n))
    pList[-1].name = p
opts.save_dof = True
opts.dataDir='.'
opts.probDir='.'
Profiling.logLevel=3
Profiling.verbose=False

In [ ]:
ns = NumericalSolution.NS_base(so, pList, nList, so.sList, opts)
ns.calculateSolution('run1')